# 说明

Tensorboard是随Tensorflow安装而安装的包（二者是独立的，需要都import进来）。Tensorboard在tf2.0中是非常简单、高效的训练过程可视化工具。下面用tf.keras高阶模块直接搭建网络进行MNIST数据集运算，展示Tensorboard的使用：

In [1]:
import tensorboard as tb
import tensorflow as tf

In [2]:
tb.__version__

'2.0.2'

### 数据导入：tf.data

In [3]:
# 加载数据集：只用训练集即可
(train_image, train_label), (test_image, test_label) = tf.keras.datasets.mnist.load_data()

In [4]:
# 维度拓展，从数组转为图像尺寸
train_image = tf.expand_dims(train_image, -1)
test_image = tf.expand_dims(test_image, -1)
train_image.shape

TensorShape([60000, 28, 28, 1])

In [5]:
# 转换数据类型：
train_image = tf.cast( train_image/255, tf.float32 )  # 归一化后，转为float32
train_label = tf.cast( train_label, tf.int32 )

test_image = tf.cast( test_image/255, tf.float32 )
test_label = tf.cast( test_label, tf.int32 )

In [6]:
# tf.data进行数据集输入：
train_dataset = tf.data.Dataset.from_tensor_slices( (train_image,train_label) )
test_dataset = tf.data.Dataset.from_tensor_slices( (test_image,test_label) )
train_dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int32)>

In [7]:
# 数据集乱序、分批次：因为后面用tf.keras直接搭网络，所以训练数据一般要.repeat()
train_dataset = train_dataset.shuffle(60000).repeat().batch(64)
test_dataset = test_dataset.shuffle(10000).batch(64)

### 网络搭建：tf.keras

In [8]:
model = tf.keras.Sequential( [
    tf.keras.layers.Conv2D( 16, [3,3], activation = 'relu', input_shape = (None, None, 1) ),
    tf.keras.layers.Conv2D( 32, [3,3], activation = 'relu' ),
    tf.keras.layers.GlobalMaxPooling2D(),  # 换一个Max池化看看（不是重点）
    tf.keras.layers.Dense(10, activation = 'softmax')
] )

In [9]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['acc']
)

### 创建tensorboard回调对象：tf.keras.callbacks.Tensorboard

注意：
- tf.keras.callbacks.xxx里面有非常多种、不同功能的回调函数，tensorboard仅是其中一种而已；这些回调函数的使用都是一样的！都是在model.fit()中加入到callbacks参数中；
- Tensorboard建议，每次生成的log文件根据“**时间**”来命名最好！

In [10]:
import os
import datetime

In [11]:
# os.path.join()路径拼接函数：所有日志文件，都放这个文件夹里 —— 因为程序不可能只运行一回！
# datetime.datetime.now()获取当前时间，strftime()自定义格式转换为字符串！
log_dir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))
log_dir, type(log_dir)

('logs\\20200516_170730', str)

In [12]:
# 参数1：日志文件存储地址，参数2：几个epoch存一个新文件
tensorboard_callback = tf.keras.callbacks.TensorBoard( log_dir, histogram_freq = 1 )  

In [13]:
# 回调函数在model.fit()中使用：
model.fit(
    train_dataset,
    epochs = 5,
    steps_per_epoch = 60000 // 64,
    validation_data = test_dataset,
    validation_steps = 10000 // 64,  # 这如果打错，运行完1个epoch就停止了！
    #callbacks = [tensorboard_callback]
)

Train for 937 steps, validate for 937 steps
Epoch 1/5
937/937 [==============================] - 36s 39ms/step - loss: 1.0427 - acc: 0.6883 - val_loss: 0.0790 - val_acc: 0.8537


### tensorboard显示：1. 在Notebook中；2. 在浏览器中

In [ ]:
# tensorboard在Notebook中显示的方法：
%load_ext tensorboard
%matplotlib inline

%tensorboard --logdir logs  # 其中logs就是那个文件夹名

In [ ]:
# tensorboard在浏览器中显示：
在cmd中输入到logs文件夹所在地址，然后输入：tensorboard --logdir logs —— 生成一个网址！